In [1]:
from datasets import load_dataset, DatasetDict, Value
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig , BertForMaskedLM
from transformers import GPT2Config, BertConfig
from transformers import Trainer, TrainingArguments
from torch.cuda import empty_cache
import wandb


wandb.init(name="New test wandb")


raw_datasets = load_dataset("tiny_shakespeare")

/home/nguyenan1/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anhduongnguyen01 (south-pole). Use `wandb login --relogin` to force relogin


Found cached dataset tiny_shakespeare (/home/nguyenan1/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 1281.88it/s]


In [2]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["text"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )
training_corpus = get_training_corpus()

In [3]:
from transformers import AutoTokenizer
# old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
old_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)


In [4]:
context_length = 128

def tokenize(element):

    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)

In [5]:
# config = GPT2Config(vocab_size=len(tokenizer),
#     n_ctx=context_length,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id)
# model = GPT2LMHeadModel(config)

config = BertConfig(vocab_size=len(tokenizer))
model = BertForMaskedLM(config)

In [6]:
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True)

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    evaluation_strategy="steps",
    eval_steps=20,
    num_train_epochs=10,
    weight_decay=0.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-3,
    fp16=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)
empty_cache()
trainer.train()

/home/nguyenan1/.local/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1570 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                 
  1%|          | 11/1570 [00:02<05:09,  5.04it/s]

{'eval_loss': 7.211300373077393, 'eval_runtime': 0.254, 'eval_samples_per_second': 444.863, 'eval_steps_per_second': 39.368, 'epoch': 0.06}


                                                 
  1%|▏         | 21/1570 [00:03<04:59,  5.17it/s]

{'eval_loss': 7.163985729217529, 'eval_runtime': 0.2528, 'eval_samples_per_second': 446.98, 'eval_steps_per_second': 39.556, 'epoch': 0.13}


                                                 
  2%|▏         | 31/1570 [00:04<04:58,  5.16it/s]

...

                                                   
 99%|█████████▉| 1561/1570 [03:44<00:01,  5.08it/s]

{'eval_loss': 7.024813175201416, 'eval_runtime': 0.2596, 'eval_samples_per_second': 435.306, 'eval_steps_per_second': 38.523, 'epoch': 9.94}


                                                   
100%|██████████| 1570/1570 [03:45<00:00,  6.95it/s]

{'eval_loss': 6.990670680999756, 'eval_runtime': 0.2567, 'eval_samples_per_second': 440.139, 'eval_steps_per_second': 38.95, 'epoch': 10.0}
{'train_runtime': 225.9665, 'train_samples_per_second': 82.888, 'train_steps_per_second': 6.948, 'train_loss': 6.5342181940746915, 'epoch': 10.0}


TrainOutput(global_step=1570, training_loss=6.5342181940746915, metrics={'train_runtime': 225.9665, 'train_samples_per_second': 82.888, 'train_steps_per_second': 6.948, 'train_loss': 6.5342181940746915, 'epoch': 10.0})